In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import nltk
import pandas as pd
import os
import re
import string 

from collections import defaultdict
from google.colab import drive
from glob import glob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from skimage import io, transform


drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ENGI 4800/data/df_converted.csv', index_col=False).dropna()[['pii','Section_content','Converted_Section_Content','Authors']]
m,n = df.shape
df.index = np.arange(m)
df.columns = ['pii','Section_content', 'Converted_Section_Content','Authors']

df_copy = df.copy()

df_splitted = df_copy

In [ ]:
splitted_texts = []
sentences_count = []
for index, row in df_splitted.iterrows():

    # add paragraph sentences count 
    content = row[2]
    
    replaced_txt = re.sub(r"([^A-Z])([.|;]\s+)", r"\1\n", content)
    splitted_txt = re.split(r"\n", replaced_txt)
    
    sentences_count.append(len(splitted_txt)) 
    splitted_texts.append(splitted_txt)
    
df_splitted.insert(1,'Sentences_count', sentences_count)
df_splitted.insert(2,'Splitted_txt', splitted_texts)
df_splitted.drop(df_splitted[df_splitted['Sentences_count'] == 1].index, inplace=True)

sentences_all = []

for index, row in df_splitted.iterrows():
    sentences_all += row[2]
df_sentences = pd.DataFrame(sentences_all, columns =['Original_txt'])

authorsId_removed = []
for index, row in df_sentences.iterrows():
    authorsId_removed.append(re.sub('#[\d]+', '', row[0]))
    
df_sentences.insert(1,'AuthorsId_removed', authorsId_removed)

In [ ]:
nltk.download('punkt', quiet=True)
nltk.download("stopwords", quiet=True)
stop_words = set(stopwords.words('english'))

punctuations = string.punctuation

punctuation_stopWord_removed = []

for index, row in df_sentences.iterrows():
    sentence = row[1]
    sentence = sentence.translate(str.maketrans('', '', string.punctuation.replace('#','')))
    filtered_sentence = [w for w in word_tokenize(sentence) if not w.lower() in stop_words]
    punctuation_stopWord_removed.append(' '.join(filtered_sentence))
    
df_sentences.insert(2,'punctuation_stopWord_removed', punctuation_stopWord_removed) 

In [ ]:
porter = PorterStemmer()
df_stem = df_sentences.copy()
stemed = []

for i, row in df_sentences.iterrows():
    sentence = df_sentences['punctuation_stopWord_removed'][i].split()
    stemed.append(' '.join([porter.stem(word) for word in sentence]))
    
df_sentences.insert(3,'stemmed', stemed) 

df_sentences_copy = df_sentences.copy()
df_sentences_copy.drop(df_sentences[(df_sentences['stemmed'] == '') ].index, inplace=True)

In [ ]:
df_review_edit = df_sentences_copy.loc[(df_sentences_copy.stemmed.str.contains('review',na=False) | 
                                   df_sentences_copy.stemmed.str.contains('revis',na=False) | 
                                   df_sentences_copy.stemmed.str.contains('edit',na=False) |
                                   df_sentences_copy.stemmed.str.contains('critic',na=False)) &
                                   ~(
                                       df_sentences_copy.stemmed.str.contains('manuscript',na=False) |
                                     df_sentences_copy.stemmed.str.contains('origin',na=False) |
                                     df_sentences_copy.stemmed.str.contains('supervis',na=False) |
                                     df_sentences_copy.stemmed.str.contains('All',na=False) |
                                     df_sentences_copy.stemmed.str.contains('draft',na=False)
                                   )
                                   ]

In [ ]:
df_draft = df_sentences_copy.loc[(df_sentences_copy.stemmed.str.contains('write manuscript',na=False) | 
                                 df_sentences_copy.stemmed.str.contains('wrote manuscript',na=False) |
                                 df_sentences_copy.stemmed.str.contains('origin draft manuscript',na=False) |
                                 df_sentences_copy.stemmed.str.contains('write paper',na=False) |
                                 df_sentences_copy.stemmed.str.contains('wrote paper',na=False) |
                                 df_sentences_copy.stemmed.str.contains('cowrot paper',na=False) |
                                 df_sentences_copy.stemmed.str.contains('cowrit paper',na=False)) &
                                 ~(
                                     df_sentences_copy.stemmed.str.contains('supervis',na=False) |
                                   df_sentences_copy.stemmed.str.contains('design',na=False) |
                                   df_sentences_copy.stemmed.str.contains('All',na=False) |
                                   df_sentences_copy.stemmed.str.contains('analyz',na=False) 
                                 )
                                 ]

In [ ]:
df_review_edit_sample = df_review_edit.sample(n=100, random_state=123).drop(columns=['AuthorsId_removed', 'punctuation_stopWord_removed','stemmed'])
df_review_edit_sample['Label'] = 'Writing - Review and Editing'
df_draft_sample = df_draft.sample(n=100, random_state=123).drop(columns=['AuthorsId_removed', 'punctuation_stopWord_removed','stemmed'])
df_draft_sample['Label'] = 'Writing - Original Draft'

In [ ]:
df_draft_sample.to_csv('/content/drive/MyDrive/ENGI 4800/data/Writing - Original Draft.csv')
df_review_edit_sample.to_csv('/content/drive/MyDrive/ENGI 4800/data/Writing - Review and Editing.csv')